# LIBRARY

In [ ]:
import matplotlib.pyplot as plt
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from datetime import date, datetime
from multiprocessing.spawn import import_main_path
from keras.preprocessing.sequence import TimeseriesGenerator
from pandas_datareader import data
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import numpy as np



# GET DATA

In [ ]:
tickers = ["ANTM.JK","ASII.JK","ICBP.JK","JSMR.JK"]

today = date.today()
# print(today)
start_date = '2010-01-01'
# end_date = '2022-08-02'

panel_data = data.DataReader(tickers,'yahoo',start_date,today)
panel_data.head(5)


In [ ]:
data_close = panel_data["Adj Close"]
data_close.head(5).round(2)
list(data_close)
data_close.columns = ['antm', 'asii', 'icbp', 'jsmr']
data_close

# Normalize data

In [ ]:
df_antm = pd.DataFrame(data_close.antm)
df_asii = pd.DataFrame(data_close.asii)
df_icbp = pd.DataFrame(data_close.icbp)
df_jsmr = pd.DataFrame(data_close.jsmr)

In [ ]:
# apply a min max scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_antm = pd.DataFrame(scaler.fit_transform(df_antm), columns = ['antm'])
df_asii = pd.DataFrame(scaler.fit_transform(df_asii), columns = ['asii'])
df_icbp = pd.DataFrame(scaler.fit_transform(df_icbp), columns = ['icbp'])
df_jsmr = pd.DataFrame(scaler.fit_transform(df_jsmr), columns = ['jsmr'])



# SLIDING WINDOW FUNCTION

In [ ]:
from numpy import array

def split_sequence(seq, n_steps):
    X,y = list(), list()
    for i in range(len(seq)):
        end_ix = i + n_steps
        if end_ix > len(seq)-1:
            break
        seq_x, seq_y =seq[i:end_ix],seq[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

# VANILA LSTM

In [ ]:
n_steps_1 = 1
n_steps_2 = 2
n_steps_3 = 3
n_steps_4 = 4
n_features = 1

## ANTM

In [ ]:
df_antm = df_antm.reset_index(drop=True)
arr_antm = df_antm.to_numpy()
flat_antm = arr_antm.flatten()
antm_X_2, antm_y_2 = split_sequence(flat_antm, n_steps_2)

# train test split
from sklearn.model_selection import train_test_split
X_train_antm_2, X_test_antm_2, y_train_antm_2, y_test_antm_2 = train_test_split(antm_X_2, antm_y_2, test_size=0.33, random_state=42)


X_train_antm_2 = X_train_antm_2.reshape(X_train_antm_2.shape[0],X_train_antm_2.shape[1],n_features)
X_test_antm_2 = X_test_antm_2.reshape(X_test_antm_2.shape[0],X_test_antm_2.shape[1],n_features)


In [ ]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_antm_n2_h1_u8_lr1_e200 = Sequential([
  LSTM(8, activation='relu',input_shape=(n_steps_2, n_features)),
  Dense(1),
])


simple_model_antm_n2_h1_u8_lr1_e200.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
  metrics=['mae'],
)

simple_model_antm_n2_h1_u8_lr1_e200.summary()


In [ ]:
smod_history_antm_n2_h1_u8_lr1_e200 = simple_model_antm_n2_h1_u8_lr1_e200.fit(X_train_antm_2, y_train_antm_2,
          validation_split=0.2,
          epochs=200,
          batch_size=batch_size,
          shuffle = True
)
#time 41.8

In [ ]:
preds_antm_n2_h1_u8_lr1_e200 = simple_model_antm_n2_h1_u8_lr1_e200.predict(X_test_antm_2)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
print("epoch 200")
print("mae score antm_n2_h1_u8_lr1_e200: "+str(mean_absolute_error(preds_antm_n2_h1_u8_lr1_e200, y_test_antm_2)))
print("r2 score antm_n2_h1_u8_lr1_e200: "+str(r2_score(preds_antm_n2_h1_u8_lr1_e200, y_test_antm_2)))
print("mape score antm_n2_h1_u8_lr1_e200: "+str(mean_absolute_percentage_error(preds_antm_n2_h1_u8_lr1_e200, y_test_antm_2)))
print("rmse score antm_n2_h1_u8_lr1_e200: "+str(np.sqrt(mean_squared_error(preds_antm_n2_h1_u8_lr1_e200, y_test_antm_2))))

# mae_antm_hl_u_lr_e = {'model_n2_h1_u8_lr1_e100':mean_absolute_error(preds_antm_n2_h1_u8_lr1_e100, y_test_antm_2),'model_n2_h1_u8_lr1_e150':mean_absolute_error(preds_antm_n2_h1_u8_lr1_e150, y_test_antm_2),'model_n2_h1_u8_lr1_e200':mean_absolute_error(preds_antm_n2_h1_u8_lr1_e200, y_test_antm_2)}

# mape_antm_hl_u_lr_e = {'model_n2_h1_u8_lr1_e100':mean_absolute_percentage_error(preds_antm_n2_h1_u8_lr1_e100, y_test_antm_2),'model_n2_h1_u8_lr1_e150':mean_absolute_percentage_error(preds_antm_n2_h1_u8_lr1_e150, y_test_antm_2),'model_n2_h1_u8_lr1_e200':mean_absolute_percentage_error(preds_antm_n2_h1_u8_lr1_e200, y_test_antm_2)}

# rmse_antm_hl_u_lr_e = {'model_n2_h1_u8_lr1_e100':np.sqrt(mean_squared_error(preds_antm_n2_h1_u8_lr1_e100, y_test_antm_2)),'model_n2_h1_u8_lr1_e150':np.sqrt(mean_squared_error(preds_antm_n2_h1_u8_lr1_e150, y_test_antm_2)),'model_n2_h1_u8_lr1_e200':np.sqrt(mean_squared_error(preds_antm_n2_h1_u8_lr1_e200, y_test_antm_2))}

# r2_antm_hl_u_lr_e = {'model_n2_h1_u8_lr1_e100':r2_score(preds_antm_n2_h1_u8_lr1_e100, y_test_antm_2),'model_n2_h1_u8_lr1_e150':r2_score(preds_antm_n2_h1_u8_lr1_e150, y_test_antm_2),'model_n2_h1_u8_lr1_e200':r2_score(preds_antm_n2_h1_u8_lr1_e200, y_test_antm_2)} 
